In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
import dotenv
from os import environ

# Importing env variables from .env file
env_file = '../.env'
dotenv.load_dotenv(env_file, override=True)

True

In [2]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate

import validators

In [3]:
import sys
sys.path.insert(0, '../src')
from config import PROMPTS
from bloggenerator import BlogGenerator
from googledocwritter import GoogleDocWritter

In [5]:
webpage = 'https://www.benandfrank.com/'
llm_summary = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)
llm_blog = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.9)
blog_topic = "importancia de usar lentes obscuros"
blog_generator = BlogGenerator(
    webpage, blog_topic,
    llm_summary, llm_blog,
    PROMPTS['summary'], PROMPTS['blog_post'],
    PROMPTS['blog_post_no_summary']
)

In [6]:
blog_content = blog_generator.generate_blog_post()

1 Retrieved the following content from webpage: 







ENVÍOS Y DEVOLUCIONES GRATIS A TODA LA REPÚBLICA

hola@benandfrank.com

Colecciones

Examen de la vista

Visítanos

Mi cuenta

Colecciones

Examen de la vista

Visítanos

Rastrea tu pedido









...

...
2 Webpage summary: La empresa Ben and Frank ofrece envíos y devoluciones gratuitas en toda la República. También ofrecen exámenes de la vista y tienen colecciones de productos. Además, se puede rastrear el pedido y visitar su sitio web para más información.


In [7]:
blog_content

'# La importancia de usar lentes obscuros\n\nCuando pensamos en lentes, generalmente pensamos en aquellos que nos ayudan a corregir nuestra visión, pero ¿has pensado en los lentes obscuros? Quizás los uses solo como un accesorio para complementar nuestro atuendo, pero en realidad tienen una función muy importante para nuestra salud ocular. En esta entrada de blog, vamos a hablar sobre la importancia de usar lentes obscuros y cómo pueden proteger nuestros ojos de los rayos solares dañinos.\n\n## ¿Por qué es importante usar lentes obscuros?\n\nEl sol emite radiación ultravioleta (UV) que puede ser perjudicial para nuestra piel y nuestros ojos, y es por eso que es importante protegernos. Los lentes obscuros pueden reducir la cantidad de radiación UV que entra en nuestros ojos, lo que nos protege de varios tipos de problemas oculares. Algunos de estos problemas incluyen:\n\n### Cataratas\n\nLas cataratas son una opacidad del cristalino del ojo y son la principal causa de ceguera en todo el

# Google Docs

In [35]:
from pathlib import Path
import os
import json

from markdown import markdown
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

In [36]:
class AuthenticationError(Exception):
    pass


In [45]:
class GoogleDocWritter():
    def __init__(self, webpage, blog_post, template, folder_id, path_to_google_creds=None):
        if path_to_google_creds:
            self.path_to_google_creds = path_to_google_creds
        else:
            self.path_to_google_creds = Path.cwd()#Path(__file__).parent.absolute()
        self.webpage = webpage
        self.blog_post = blog_post
        self.template = template
        self.folder_id = folder_id

        self.gauth = None
    
    def write_creds(self):
        """ Write service account credentials to disk based on environment variables."""
        credentials = {
            "type": "service_account",
            "project_id": os.getenv("GOOGLE_PROJECT_ID"),
            "private_key_id": os.getenv("GOOGLE_PRIVATE_KEY_ID"),
            "private_key": os.getenv("GOOGLE_PRIVATE_KEY"),
            "client_email": os.getenv("GOOGLE_CLIENT_EMAIL"),
            "client_id": os.getenv("GOOGLE_CLIENT_ID"),
            "auth_uri": "https://accounts.google.com/o/oauth2/auth",
            "token_uri": "https://oauth2.googleapis.com/token",
            "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
            "client_x509_cert_url": os.getenv("GOOGLE_CLIENT_X509_CERT_URL")
        }
        with open(os.path.join(self.path_to_google_creds, os.getenv("GOOGLE_CREDENTIALS_FILENAME")), "w+") as f:
            f.write(json.dumps(credentials))
    
    def authenticate(self):
        secret_file_path = os.path.join(self.path_to_google_creds, os.getenv("GOOGLE_CREDENTIALS_FILENAME"))
        if not os.path.exists(secret_file_path):
            ValueError('Can not find secret file in the provided path')

        settings = {
            "client_config_backend": "service",
            "service_config": {
                "client_json_file_path": secret_file_path,
            }
        }
        
        self.gauth = GoogleAuth(settings=settings)
        self.gauth.ServiceAuth()
    
    def generate_html_post(self):
        htmlpost = markdown(self.blog_post)
        return self.template.format(blog_text=htmlpost)

    def create_file(self, htmldoc):
        if self.gauth:
            drive = GoogleDrive(self.gauth)
            gdoc = drive.CreateFile(
                {
                    'parents': [{'id': self.folder_id}],
                    "title": f"Demo - {self.webpage}",
                    "mimeType": "application/vnd.openxmlformats-officedocument.wordprocessingml.document"
                }
            )
            gdoc.SetContentString(htmldoc)
            gdoc.Upload()
            return gdoc["alternateLink"]
        else:
            raise AuthenticationError("Missing authentication")
    
    def upload_blog_post(self):
        self.write_creds()
        self.authenticate()
        htmldoc = self.generate_html_post()
        doc_url = self.create_file(htmldoc)
        return doc_url 

#TODO: Check blogpost size and ask llm to rate blog post
        

In [46]:
template = """<h1></h1>
<p> [Demo] </p>
<p><em>¡Hola! Muchas gracias por tu interés en Blogggs. Este artículo es un ejemplo del trabajo 
que realizamos y una probadita del tipo de posts que podemos hacer para tu marca.</em></p>
<br>
<p><em>Tómalo como un regalo y siéntete libre de publicarlo en tu blog</em> 😉</p>
<br>
<hr>
<br>
{blog_text}
<br>
<hr>
<br>
<br>
<p><strong>¿Te gustó lo que leíste y te gustaría que empezáramos a colaborar contigo?</strong></p>
<br>
<p><em>Puedes elegir el plan que más te acomode a través de <a href="http://www.blogggs.com/comenzar">www.blogggs.com/comenzar</a>.</em></p>
<p><em>Si tienes alguna duda antes de contratar, escríbenos a <a href="mailto:agustin@blogggs.com">agustin@blogggs.com</a>.</em></p>
<br>
<p><em>¡Saludos y esperamos escuchar de ti muy pronto!</em></p>
<br>
<p><em>Si NO te gustó el contenido, agradeceríamos mucho tu feedback a través de <a href="https://forms.gle/R9FFLyKVEui5dCZG6">esta liga</a>.</em></p>"""
folder_id = '1bHUB2-YELFEl690aDzhs_zzOTYntAiTk'

In [47]:
google_doc_writter = GoogleDocWritter(webpage, blog_content, template, folder_id)

In [48]:
google_doc_writter.upload_blog_post()

'https://docs.google.com/document/d/16G-IX7O1bmcHxT0H4VcEySoSOcFGgX58/edit?usp=drivesdk&ouid=108083632963361298181&rtpof=true&sd=true'

In [7]:
import os
def write_creds():
    """ Write service account credentials to disk based on environment variables."""
    credentials = {
        "type": "service_account",
        "project_id": os.getenv("GOOGLE_PROJECT_ID"),
        "private_key_id": os.getenv("GOOGLE_PRIVATE_KEY_ID"),
        "private_key": os.getenv("GOOGLE_PRIVATE_KEY"),
        "client_email": os.getenv("GOOGLE_CLIENT_EMAIL"),
        "client_id": os.getenv("GOOGLE_CLIENT_ID"),
        "auth_uri": "https://accounts.google.com/o/oauth2/auth",
        "token_uri": "https://oauth2.googleapis.com/token",
        "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
        "client_x509_cert_url": os.getenv("GOOGLE_CLIENT_X509_CERT_URL")
    }
    with open(os.path.join(here(), os.getenv("GOOGLE_CREDENTIALS_FILENAME")), "w+") as f:
        f.write(json.dumps(credentials))

In [8]:
from pyprojroot import here
import json

In [114]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive


write_creds()

secret_file = os.path.join(here(), os.getenv("GOOGLE_CREDENTIALS_FILENAME"))

settings = {
    "client_config_backend": "service",
    "service_config": {
        "client_json_file_path": secret_file,
    }
}


gauth = GoogleAuth(settings=settings)
gauth.ServiceAuth()
drive = GoogleDrive(gauth)

In [10]:
from oauth2client.service_account import ServiceAccountCredentials

In [11]:
gauth = GoogleAuth()
scope = ["https://www.googleapis.com/auth/drive"]
gauth.auth_method = 'service'
gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name(secret_file, scope)
drive = GoogleDrive(gauth)

In [115]:
template = """<h1></h1>
<p> [Demo] </p>
<p><em>¡Hola! Muchas gracias por tu interés en Blogggs. Este artículo es un ejemplo del trabajo 
que realizamos y una probadita del tipo de posts que podemos hacer para tu marca.</em></p>
<br>
<p><em>Tómalo como un regalo y siéntete libre de publicarlo en tu blog</em> 😉</p>
<br>
<hr>
<br>
{blog_text}
<br>
<hr>
<br>
<br>
<p><strong>¿Te gustó lo que leíste y te gustaría que empezáramos a colaborar contigo?</strong></p>
<br>
<p><em>Puedes elegir el plan que más te acomode a través de <a href="http://www.blogggs.com/comenzar">www.blogggs.com/comenzar</a>.</em></p>
<p><em>Si tienes alguna duda antes de contratar, escríbenos a <a href="mailto:agustin@blogggs.com">agustin@blogggs.com</a>.</em></p>
<br>
<p><em>¡Saludos y esperamos escuchar de ti muy pronto!</em></p>
<br>
<p><em>Si NO te gustó el contenido, agradeceríamos mucho tu feedback a través de <a href="https://forms.gle/R9FFLyKVEui5dCZG6">esta liga</a>.</em></p>"""

In [106]:
from markdown import markdown

htmldoc = markdown(blog_content)

In [116]:
htmldoc = template.format(blog_text=htmldoc)

In [117]:
gdoc = drive.CreateFile(
    {
        'parents': [{'id': '1bHUB2-YELFEl690aDzhs_zzOTYntAiTk'}],
        "title": f"Demo - {webpage}",
        "mimeType": "application/vnd.openxmlformats-officedocument.wordprocessingml.document"
    }
)
gdoc.SetContentString(htmldoc)
gdoc.Upload()

In [118]:
gdoc["alternateLink"]

'https://docs.google.com/document/d/1mz_hzFA8-r9MF2JZm1mKEoYuIYX818-q/edit?usp=drivesdk&ouid=108083632963361298181&rtpof=true&sd=true'

In [4]:
from config import FOLDER_ID

In [5]:
google_doc_writter = GoogleDocWritter(
    webpage="test.com", blog_post='# Testing',
    template="""<h1></h1><p>The name of the post is <\p> <br> {blog_text}""",
    folder_id=FOLDER_ID
    )

google_doc_writter.write_creds()
google_doc_writter.authenticate()
html_post = google_doc_writter.generate_html_post()
doc_url = google_doc_writter.create_file(html_post)

In [6]:
doc_url

'https://docs.google.com/document/d/1Pm2iouSfx9kF8HAvkMJLBbSqKks_npXh/edit?usp=drivesdk&ouid=108083632963361298181&rtpof=true&sd=true'

In [7]:
google_doc_writter.drive

In [9]:
file_list = google_doc_writter.drive.ListFile({'q': "'root' in parents"}).GetList()

In [15]:
file_list = google_doc_writter.drive.ListFile(
    {
        "q": '"{}" in parents and title="{}" and trashed=false'.format(
            FOLDER_ID, 'Demo - test.com'
        )
    }
).GetList()

In [16]:
file_list

[GoogleDriveFile({'kind': 'drive#file', 'userPermission': {'id': 'me', 'type': 'user', 'role': 'owner', 'kind': 'drive#permission', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1Pm2iouSfx9kF8HAvkMJLBbSqKks_npXh/permissions/me', 'etag': '"Qi--ihOn0xWvSs-dIFsu9zpQdPA"', 'pendingOwner': False}, 'fileExtension': 'com', 'md5Checksum': '4efe6f04c7a233c39f0b60996c3ec5fd', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1Pm2iouSfx9kF8HAvkMJLBbSqKks_npXh', 'ownerNames': ['blog-generation@blog-content-creation.iam.gserviceaccount.com'], 'lastModifyingUserName': 'blog-generation@blog-content-creation.iam.gserviceaccount.com', 'editable': True, 'writersCanShare': True, 'downloadUrl': 'https://www.googleapis.com/drive/v2/files/1Pm2iouSfx9kF8HAvkMJLBbSqKks_npXh?alt=media&source=downloadUrl', 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'parents': [{'selfLink': 'https://www.googleapis.com/drive/v2/files/1Pm2iouSfx9kF8HAvkMJLBbSqKks_npXh/paren

In [17]:
import pandas as pd

In [22]:
blog_post_info = pd.read_csv('../src/Demo Submissions Blogggs - Sheet1.csv')
llm_summary = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)
llm_blog = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.9)
blog_post_info.rename(
    columns={
        'Tema del artículo': 'tema_articulo',
        'Página web': 'pagina_web'
        }, inplace=True)
for row in blog_post_info.itertuples():
    blog_generator = BlogGenerator(
        row.pagina_web, row.tema_articulo,
        llm_summary, llm_blog,
        PROMPTS['summary'], PROMPTS['blog_post'],
        PROMPTS['blog_post_no_summary']
    )
    

Cuidado y mantenimiento del calzado
Moda para tallas extra y body positive
Buenas, 
 

 Somos una empresa de sustainable fashion basado en los cabos. Me gustaria concretar una llamada con ustedes y mis socios para ver si nos pueden ayudar en nuestro blog, sitio web, redes sociales, etc. Me pueden contactar por whatsapp!
 

 Lic. Asher Francis
Consejos para comprar una casa
Marketing digital
Tedencias de fashion curvy
10 básicos para un picnic exitoso
Beneficios de tomar matcha y/o una receta de mstcha
Cómo afectan la corrupción y la falta de rendición de cuentas a la desigualdad social
